In [8]:
import pandas as pd
import numpy as np
import os
import glob
from functools import reduce
from datetime import date, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import plotly.express as px
import altair as alt
import ipywidgets as widgets
import IPython.display
from IPython.display import display, clear_output


In [9]:
df1 = pd.read_csv('Data/covid19_vaccination_data_US_full.csv')

In [10]:
df1.head()

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Administered_Dose1_Recip_5PlusPop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Administered_5Plus,Admin_Per_100k_5Plus,Distributed_Per_100k_5Plus,Series_Complete_Moderna_5Plus,Series_Complete_Pfizer_5Plus,Series_Complete_Janssen_5Plus,Series_Complete_Unk_Manuf_5Plus
0,11/20/2021,46,US,567081775,27398400,217592840,322090535,0,170803,200015,...,73.6,196119179.0,62.8,449912252.0,144080.0,181603.0,71609210.0,108643416.0,15741770.0,124783.0
1,11/20/2021,46,CO,9948285,447600,3794340,5706345,0,172751,201543,...,74.6,3612834.0,66.6,8365665.0,154162.0,183327.0,1353761.0,1970893.0,286363.0,1817.0
2,11/20/2021,46,WA,13809255,681200,5172680,7955375,0,181345,212420,...,76.3,4920505.0,68.7,11076182.0,154729.0,192909.0,1786017.0,2712390.0,419439.0,2659.0
3,11/20/2021,46,DE,1848455,81400,733460,1033595,0,189826,220105,...,75.5,593369.0,64.6,1386361.0,150848.0,201128.0,215854.0,324584.0,52397.0,534.0
4,11/20/2021,46,MT,1625215,96900,677600,850715,0,152063,177116,...,62.6,551251.0,54.7,1295704.0,128590.0,161292.0,224266.0,276683.0,49946.0,356.0


In [11]:
len(df1)

22168

In [18]:

df1 = df1[df1.Location != "US"]
province_list = np.unique(df1['Location'].values)
# print(province_list)

prov = province_list[2]
comp_prov = province_list[5]

dropdown_state = widgets.Dropdown(options = sorted(province_list), value=prov, description='State 1:')
dropdown_comp_state = widgets.Dropdown(options = sorted(province_list), value=comp_prov, description='State 2:')


sorted_df = df1.copy()
sorted_df["Date"] = pd.to_datetime(sorted_df["Date"])
# sorted_df["order"] = sorted_df["Location"].map({"US": 1, "CO": 2}).fillna(3)
if(comp_prov is None):
    sorted_df["order"] = sorted_df["Location"].map({prov: 1}).fillna(2)
else:
    sorted_df["order"] = sorted_df["Location"].map({prov: 1, comp_prov: 2}).fillna(3)
sorted_df.sort_values(by=["order", "Date"], ascending=False, inplace=True)

fig = px.line(sorted_df, 
        x="Date", 
        y="Administered", 
        color="Location", 
        labels={
             "Date": "Date Administered",
             "Administered": "Vaccine Doses Administered",
             "species": "Species of Iris"
         }, 
        width=800, height=900,
        title="Admininstration of Covid Vaccines Across Various Regions in the US")

fig.update_traces({"line":{"color":"lightgrey", "width":2}})

fig.update_traces(patch={"line":{"color":"blue", "width":3}}, 
                  selector={"legendgroup":prov})

if(comp_prov is not None):
    fig.update_traces(patch={"line":{"color":"red", "width":3}}, 
                      selector={"legendgroup":comp_prov})

fig.update_layout(title_text='Admininstration of Covid Vaccines Across Various Regions in the US', title_x=0.5,
                showlegend=True,
                yaxis_range=[0,58500000],
                yaxis={"visible":True})


# loc_dict_list = []
# for i in range(len(province_list)):
#     d = {"args":, "label":province_list[i], "method":"update"}
#     loc_dict_list.append(d)

# updatemenus=list([
#     dict(
#         buttons=loc_dict_list,
#         direction = 'down',
#         pad = {'r': 10, 't': 10},
#         showactive = True,
#         x = 0.1,
#         xanchor = 'left',
#         y = 1.1,
#         yanchor = 'top'
#     ),
# ])


def dropdown_state_eventhandler(change):
    """
    Eventhandler for the state dropdown widget
    """
    display(input_widgets)
    global prov, comp_prov
    prov = change.new
    
    sorted_df = df1.copy()
    sorted_df["Date"] = pd.to_datetime(sorted_df["Date"])
    # sorted_df["order"] = sorted_df["Location"].map({"US": 1, "CO": 2}).fillna(3)
    if(comp_prov is None):
        sorted_df["order"] = sorted_df["Location"].map({prov: 1}).fillna(2)
    else:
        sorted_df["order"] = sorted_df["Location"].map({prov: 1, comp_prov: 2}).fillna(3)
    sorted_df.sort_values(by=["order", "Date"], ascending=False, inplace=True)

    fig = px.line(sorted_df, 
            x="Date", 
            y="Administered", 
            color="Location", 
            labels={
                 "Date": "Date Administered",
                 "Administered": "Vaccine Doses Administered",
                 "species": "Species of Iris"
             }, 
            width=800, height=900,
            title="Admininstration of Covid Vaccines Across Various Regions in the US")

    fig.update_traces({"line":{"color":"lightgrey", "width":2}})

    fig.update_traces(patch={"line":{"color":"blue", "width":3}}, 
                      selector={"legendgroup":prov})

    if(comp_prov is not None):
        fig.update_traces(patch={"line":{"color":"red", "width":3}}, 
                          selector={"legendgroup":comp_prov})

    fig.update_layout(title_text='Admininstration of Covid Vaccines Across Various Regions in the US', title_x=0.5,
                    showlegend=True,
                    yaxis_range=[0,58500000],
                    yaxis={"visible":True})

    fig.show()
    IPython.display.clear_output(wait=True)            

    
def dropdown_comp_state_eventhandler(change):
    """
    Eventhandler for the state dropdown widget
    """
    display(input_widgets)
    global prov, comp_prov
    comp_prov = change.new
    sorted_df = df1.copy()
    sorted_df["Date"] = pd.to_datetime(sorted_df["Date"])
    # sorted_df["order"] = sorted_df["Location"].map({"US": 1, "CO": 2}).fillna(3)
    if(comp_prov is None):
        sorted_df["order"] = sorted_df["Location"].map({prov: 1}).fillna(2)
    else:
        sorted_df["order"] = sorted_df["Location"].map({prov: 1, comp_prov: 2}).fillna(3)
    sorted_df.sort_values(by=["order", "Date"], ascending=False, inplace=True)

    fig = px.line(sorted_df, 
            x="Date", 
            y="Administered", 
            color="Location", 
            labels={
                 "Date": "Date Administered",
                 "Administered": "Vaccine Doses Administered",
                 "species": "Species of Iris"
             }, 
            width=800, height=900,
            title="Admininstration of Covid Vaccines Across Various Regions in the US")

    fig.update_traces({"line":{"color":"lightgrey", "width":2}})

    fig.update_traces(patch={"line":{"color":"blue", "width":3}}, 
                      selector={"legendgroup":prov})

    if(comp_prov is not None):
        fig.update_traces(patch={"line":{"color":"red", "width":3}}, 
                          selector={"legendgroup":comp_prov})

    fig.update_layout(title_text='Admininstration of Covid Vaccines Across Various Regions in the US', title_x=0.5,
                    showlegend=True,
                    yaxis_range=[0,58500000],
                    yaxis={"visible":True})

    fig.show()
    IPython.display.clear_output(wait=True)            

    
dropdown_state.observe(dropdown_state_eventhandler, names='value')
dropdown_comp_state.observe(dropdown_comp_state_eventhandler, names='value')

input_widgets = widgets.VBox([dropdown_state, dropdown_comp_state])
display(input_widgets)
fig.show()
IPython.display.clear_output(wait=True)    


